In [1]:
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation

from explaneat.core.neuralneat import NeuralNeat as NeuralNeat

import random

import torch
import torch.nn as nn
import torch.optim as optim

import neat

from explaneat.core.utility import MethodTimer


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter

from explaneat.core.neuralneat import NeuralNeat as nneat

import torch.optim as optim

from torch.functional import F

import pprint

pp = pprint.PrettyPrinter(indent=4)







In [2]:
N_GENERATIONS = 50
N_EPOCHS = 5

In [3]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [4]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [5]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(10, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

config = base_config
saveLocation = './'
maxNGenerations = N_GENERATIONS
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = N_EPOCHS)

g = p.best_genome


The function - Backprop everything - has just started at 1644201509.189793
The function - Backprop everything - took 0.0034117698669433594 seconds to complete
The function - generationStart - has just started at 1644201509.193773

 ****** Running generation 0 ****** 

The function - generationStart - took 3.1948089599609375e-05 seconds to complete
The function - pre_backprop - has just started at 1644201509.193942
The function - pre_backprop - took 9.179115295410156e-05 seconds to complete
The function - backprop - has just started at 1644201509.1940448
about to start backprop with 5 epochs
mean improvement: -0.0016957445211506583
best improvement: -0.002604102747592596
best loss: 0.5006431066353582
The function - backprop - took 0.1708049774169922 seconds to complete
The function - post_backprop - has just started at 1644201509.364867
The function - post_backprop - took 1.6927719116210938e-05 seconds to complete
The function - evaluate fitness - has just started at 1644201509.36494
{1

In [6]:
print(g)

Key: 1075
Fitness: 1.6452809572219849
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.02307499607111329, response=1.0, activation=sigmoid, aggregation=sum)
	82 DefaultNodeGene(key=82, bias=-2.0709136943466557, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.9217690055205894, enabled=False)
	DefaultConnectionGene(key=(-2, 82), weight=-2.010701063562547, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=1.8718137376241872, enabled=False)
	DefaultConnectionGene(key=(-1, 82), weight=1.7603203072876419, enabled=True)
	DefaultConnectionGene(key=(82, 0), weight=0.7489711708768533, enabled=True)


In [7]:
net = nneat(g, p.config, criterion=nn.BCEWithLogitsLoss())


In [8]:
net.genome.connections
# Error is in inputs not being updated with extra info from skip layers

{(-1, 0): <neat.genes.DefaultConnectionGene at 0x7fcb38902b20>,
 (-2, 0): <neat.genes.DefaultConnectionGene at 0x7fcb38902b80>,
 (-1, 82): <neat.genes.DefaultConnectionGene at 0x7fcb38902be0>,
 (82, 0): <neat.genes.DefaultConnectionGene at 0x7fcb38902c40>,
 (-2, 82): <neat.genes.DefaultConnectionGene at 0x7fcb38902ca0>}

In [9]:
net.forward(torch.tensor( xor_inputs[:10], dtype=torch.float64) ) [:10]

tensor([[0.6017],
        [0.5219],
        [0.5176],
        [0.6272],
        [0.5665],
        [0.5024],
        [0.5081],
        [0.5226],
        [0.5002],
        [0.6283]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [10]:
net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )[:20]

tensor([[0.6017],
        [0.5219],
        [0.5176],
        [0.6272],
        [0.5665],
        [0.5024],
        [0.5081],
        [0.5226],
        [0.5002],
        [0.6283],
        [0.5210],
        [0.5359],
        [0.5958],
        [0.5189],
        [0.5354],
        [0.5183],
        [0.4981],
        [0.5059],
        [0.5316],
        [0.5406]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [11]:
# net.optimiser.zero_grad()

In [12]:
# net.optimise(xor_inputs, xor_outputs)

In [13]:
optimiser = optim.Adadelta(net.parameters())

In [14]:
net.weights

{2: Parameter containing:
 tensor([[0.0000],
         [0.0000],
         [0.7490]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[ 1.7603],
         [-2.0107]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [15]:
optimizer = optimiser
losses = []
for i in range(10000):
    preds = net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )
    loss = F.mse_loss(preds, torch.tensor( xor_outputs, dtype=torch.float64)).sqrt()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss)  
losses[-10:]


[tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3629, dtype=torch.float64, grad_fn=<SqrtBackward0>)]

In [16]:
preds[:10]

tensor([[0.9360],
        [0.1148],
        [0.1070],
        [0.9314],
        [0.6803],
        [0.4185],
        [0.2217],
        [0.1092],
        [0.5906],
        [0.9314]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [17]:
net.weights

{2: Parameter containing:
 tensor([[-4.4652],
         [ 4.7558],
         [14.0788]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[ 3.0281],
         [-3.2995]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [18]:
net.layers

{2: {'nodes': {0: {'depth': 2,
    'output_ids': [],
    'input_ids': [-1, -2, 82],
    'depths': [],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 1],
  'input_shape': 3,
  'weights_shape': (3, 1),
  'out_weights': [],
  'bias': [-0.02307499607111329],
  'in_weights': [[0]],
  'input_weights': array([[0.        ],
         [0.        ],
         [0.74897117]]),
  'input_map': {(82, 0): (2, 0)}},
 1: {'nodes': {82: {'depth': 1,
    'output_ids': [0],
    'input_ids': [-1, -2],
    'depths': [],
    'output_layers': [2],
    'needs_skip': False,
    'id': 82,
    'input_layers': [0, 0],
    'skip_layer_input': False,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'layer_type': 'CONNECTED',
  'input_layers': [0],
  'input_shape': 2,
  'weights_shape': (2

In [19]:
net.node_tracker[0]['depth']

2

In [20]:


reached_nodes = []

node_stack = []

# Instantiate stack with depth==0 nodes
for node_id, node in net.node_tracker.items():
    if node_id < 0:
        node_stack.append(node_id)

while len(node_stack) > 0:
    reached_nodes.append(node_stack[0])
    for node_id in net.node_tracker[node_stack[0]]['output_ids']:
        node_stack.append(node_id)
    del(node_stack[0])

if True:
    print("the reached nodes are")
    print(reached_nodes)


for node_id in net.node_tracker:
    if not node_id in reached_nodes:
        print(reached_nodes)
        print(net.node_tracker)
        print("I can't reach this node {}".format(node_id))

the reached nodes are
[-1, -2, 0, 82, 0, 82, 0, 0]


In [21]:
print(net.genome)

Key: 1075
Fitness: 1.6452809572219849
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.02307499607111329, response=1.0, activation=sigmoid, aggregation=sum)
	82 DefaultNodeGene(key=82, bias=-2.0709136943466557, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.9217690055205894, enabled=False)
	DefaultConnectionGene(key=(-2, 82), weight=-2.010701063562547, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=1.8718137376241872, enabled=False)
	DefaultConnectionGene(key=(-1, 82), weight=1.7603203072876419, enabled=True)
	DefaultConnectionGene(key=(82, 0), weight=0.7489711708768533, enabled=True)


In [22]:
net.node_mapping.layers

{2: {'nodes': {0: {'depth': 2,
    'output_ids': [],
    'input_ids': [82],
    'depths': [2, 2],
    'on_path_to_output': True,
    'on_path_to_input': False,
    'is_input': False,
    'is_output': True,
    'is_valid': True,
    'output_layers': [],
    'input_layers': [1],
    'skips_in': False,
    'skips_out': False,
    'layer': 2,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'layer_activation': 'Sigmoid',
  'input_layers': [1],
  'input_shape': 1,
  'output_shape': 1,
  'weights_shape': (1, 1),
  'input_map': {},
  'shape': (1, 1)},
 1: {'nodes': {82: {'depth': 1,
    'output_ids': [0],
    'input_ids': [-1, -2],
    'depths': [1, 1],
    'on_path_to_output': True,
    'on_path_to_input': True,
    'is_input': False,
    'is_output': False,
    'is_valid': True,
    'output_layers': [2],
    'input_layers': [0, 0],
    'skips_in': False,
    'skips_out': False,
    'layer': 1,
    'layer_index': 0}},
  'is_output_laye

In [23]:
for layer_id in sorted (net.node_mapping.layers.keys()):
    layer = net.node_mapping.layers[layer_id]
    print(layer_id)
    print(layer['input_layers'])
    print(layer['shape'])
    print("----")

0
[]
(0, 2)
----
1
[0]
(2, 1)
----
2
[1]
(1, 1)
----


In [24]:
pp.pprint(net.node_mapping.connection_map)

{(-2, 82): (1, 1, 0, True), (-1, 82): (1, 0, 0, True), (82, 0): (2, 0, 0, True)}


In [25]:
net.node_mapping.connection_map

{(82, 0): (2, 0, 0, True),
 (-1, 82): (1, 0, 0, True),
 (-2, 82): (1, 1, 0, True)}

In [26]:
pp.pprint(net.genome.connections)

{   (-2, 0): <neat.genes.DefaultConnectionGene object at 0x7fcb38902b80>,
    (-2, 82): <neat.genes.DefaultConnectionGene object at 0x7fcb38902ca0>,
    (-1, 0): <neat.genes.DefaultConnectionGene object at 0x7fcb38902b20>,
    (-1, 82): <neat.genes.DefaultConnectionGene object at 0x7fcb38902be0>,
    (82, 0): <neat.genes.DefaultConnectionGene object at 0x7fcb38902c40>}


In [32]:
for connection_id, connection in net.genome.connections.items():
    print(connection_id)
    print(connection.enabled)

(-1, 0)
False
(-2, 0)
False
(-1, 82)
True
(82, 0)
True
(-2, 82)
True
